In [1]:
%load_ext jupyter_black

In [2]:
import duckdb
import editdistance
import polars as pl
from tldextract import extract

Preview input

In [3]:
!head -n 1 ctl_records_sample.jsonlines | jq .

{
  "data": {
    "cert_index": 693605056,
    "cert_link": "http://ct.googleapis.com/logs/argon2019/ct/v1/get-entries?start=693605056&end=693605056",
    "chain": [
      {
        "as_der": "MIIEkjCCA3qgAwIBAgIQCgFBQgAAAVOFc2oLheynCDANBgkqhkiG9w0BAQsFADA/MSQwIgYDVQQKExtEaWdpdGFsIFNpZ25hdHVyZSBUcnVzdCBDby4xFzAVBgNVBAMTDkRTVCBSb290IENBIFgzMB4XDTE2MDMxNzE2NDA0NloXDTIxMDMxNzE2NDA0NlowSjELMAkGA1UEBhMCVVMxFjAUBgNVBAoTDUxldCdzIEVuY3J5cHQxIzAhBgNVBAMTGkxldCdzIEVuY3J5cHQgQXV0aG9yaXR5IFgzMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAnNMM8FrlLke3cl03g7NoYzDq1zUmGSXhvb418XCSL7e4S0EFq6meNQhY7LEqxGiHC6PjdeTm86dicbp5gWAf15Gan/PQeGdxyGkOlZHP/uaZ6WA8SMx+yk13EiSdRxta67nsHjcAHJyse6cF6s5K671B5TaYucv9bTyWaN8jKkKQDIZ0Z8h/pZq4UmEUEz9l6YKHy9v6Dlb2honzhT+Xhq+w3Brvaw2VFn3EK6BlspkENnWAa6xK8xuQSXgvopZPKiAlKQTGdMDQMc2PMTiVFrqoM7hD8bEfwzB/onkxEz0tNvjj/PIzark5McWvxI0NHWQWM6r6hCm21AvA2H3DkwIDAQABo4IBfTCCAXkwEgYDVR0TAQH/BAgwBgEB/wIBADAOBgNVHQ8BAf8EBAMCAYYwfwYIKwYBBQUHAQEEczBxMDIGCCsGAQUFBzABhiZodHRwOi8vaXNyZy50cnVzdGl

Import data into DuckDB

In [4]:
con = duckdb.connect("data.duckdb")

In [5]:
con.execute(
    """
    CREATE TABLE ctl_records AS 
    SELECT * FROM read_ndjson_auto('ctl_records_sample.jsonlines', union_by_name=true)
    """
)

Verify imported correctly

In [6]:
con.sql("SELECT COUNT(*) FROM ctl_records")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      4000000 │
└──────────────┘

In [7]:
con.sql("DESCRIBE ctl_records")

┌──────────────┬───────────────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name  │                          column_type                          │  null   │   key   │ default │  extra  │
│   varchar    │                            varchar                            │ varchar │ varchar │ varchar │ varchar │
├──────────────┼───────────────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ data         │ STRUCT(cert_index BIGINT, cert_link VARCHAR, "chain" STRUCT…  │ YES     │ NULL    │ NULL    │ NULL    │
│ message_type │ VARCHAR                                                       │ YES     │ NULL    │ NULL    │ NULL    │
└──────────────┴───────────────────────────────────────────────────────────────┴─────────┴─────────┴─────────┴─────────┘

In [8]:
con.sql("SELECT message_type, COUNT(*) FROM ctl_records GROUP BY message_type")

┌────────────────────┬──────────────┐
│    message_type    │ count_star() │
│      varchar       │    int64     │
├────────────────────┼──────────────┤
│ certificate_update │      4000000 │
└────────────────────┴──────────────┘

Notice that all `message_type` are the same, would like to unnest `data` at least 1 layer

In [9]:
!head -n 1 ctl_records_sample.jsonlines | jq '.data | keys'

[
  "cert_index",
  "cert_link",
  "chain",
  "leaf_cert",
  "seen",
  "source",
  "update_type"
]


In [10]:
con.execute(
    """
    CREATE TABLE cert_data AS
    SELECT 
        data.cert_index,
        data.cert_link,
        data.chain,
        data.leaf_cert,
        data.seen,
        data.source,
        data.update_type
    FROM ctl_records
    """
)

In [11]:
con.execute("DROP TABLE ctl_records")

In [12]:
con.sql("DESCRIBE cert_data")

┌─────────────┬────────────────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │                          column_type                           │  null   │   key   │ default │  extra  │
│   varchar   │                            varchar                             │ varchar │ varchar │ varchar │ varchar │
├─────────────┼────────────────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ cert_index  │ BIGINT                                                         │ YES     │ NULL    │ NULL    │ NULL    │
│ cert_link   │ VARCHAR                                                        │ YES     │ NULL    │ NULL    │ NULL    │
│ chain       │ STRUCT(as_der VARCHAR, "extensions" STRUCT(authorityInfoAcce…  │ YES     │ NULL    │ NULL    │ NULL    │
│ leaf_cert   │ STRUCT(all_domains VARCHAR[], as_der VARCHAR, "extensions" S…  │ YES     │ NULL    │ NULL    │ NULL    │
│ seen        │ DOUBLE          

In [13]:
con.sql("SELECT update_type, COUNT(*) FROM cert_data GROUP BY update_type")

┌─────────────────┬──────────────┐
│   update_type   │ count_star() │
│     varchar     │    int64     │
├─────────────────┼──────────────┤
│ X509LogEntry    │      1882291 │
│ PrecertLogEntry │      2117709 │
└─────────────────┴──────────────┘

From here we'll load only a few things we'll need into a Polars dataframe.

We're extracting the common name (`CN`) from the certificate and the issuer, since that is what is least likely to be null.

I'm also grabbing the first entry in the `chain` field, because from what I understand, the first in the chain is the one that directly issued the certificate.

Note that nested arrays in DuckDB are 1-indexed.

In [14]:
df = con.execute(
    """
    SELECT
        leaf_cert.fingerprint,
        leaf_cert.all_domains,
        leaf_cert.subject.CN,
        chain[1].subject.CN AS issuer
    FROM cert_data
    """
).pl()

In [15]:
df.count()

fingerprint,all_domains,CN,issuer
u32,u32,u32,u32
4000000,4000000,3982630,3999942


We see that a few of our common names from our certs and our issuers are null

In [16]:
con.sql(
    "SELECT DISTINCT leaf_cert.subject FROM cert_data WHERE leaf_cert.subject.CN IS NULL"
)

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                       subject                                                        │
│           struct(c varchar, cn varchar, l varchar, o varchar, ou varchar, st varchar, aggregated varchar)            │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ {'C': GB, 'CN': NULL, 'L': London, 'O': Google Certificate Transparency, 'OU': NULL, 'ST': NULL, 'aggregated': /C=…  │
│ {'C': NULL, 'CN': NULL, 'L': NULL, 'O': NULL, 'OU': NULL, 'ST': NULL, 'aggregated': /}                               │
│ {'C': NULL, 'CN': NULL, 'L': NULL, 'O': NULL, 'OU': PositiveSSL Multi-Domain, 'ST': NULL, 'aggregated': /OU=Positi…  │
│ {'C': US, 'CN': NULL, 'L': Mountain View, 'O': Google LLC, 'OU': NULL, 'ST': California, 'aggregated': /C=US/L=Mou…  │
└───────────────────────────────

In [17]:
con.sql(
    "select distinct chain[1].subject FROM cert_data WHERE chain[1].subject.CN IS NULL"
)

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                 ("chain"[1]).subject                                                 │
│           struct(c varchar, cn varchar, l varchar, o varchar, ou varchar, st varchar, aggregated varchar)            │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ {'C': TW, 'CN': NULL, 'L': NULL, 'O': Chunghwa Telecom Co., Ltd., 'OU': Public Certification Authority - G2, 'ST':…  │
│ {'C': GB, 'CN': NULL, 'L': London, 'O': Google Certificate Transparency (Precert Signing), 'OU': NULL, 'ST': NULL,…  │
│ {'C': TW, 'CN': NULL, 'L': NULL, 'O': 行政院, 'OU': 政府憑證管理中心, 'ST': NULL, 'aggregated': /C=TW/O=行政院/OU=…  │
└──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

We'll drop the rows that have null values.

It's also a good time to de-dupe our data, so our counts don't get skewed later on.

In [18]:
df = df.drop_nulls()
df = df.unique()
df.count()

fingerprint,all_domains,CN,issuer
u32,u32,u32,u32
2928605,2928605,2928605,2928605


Proof that fingerprints are unique after de-duplication:

In [19]:
df.select("fingerprint").n_unique() == df.shape[0]

True

In [20]:
df.group_by("fingerprint").agg(pl.len().alias("count")).sort(
    by=pl.col("count"), descending=True
)

fingerprint,count
str,u32
"""32:C6:0A:31:A3:BE:80:08:C2:A8:…",1
"""33:3F:73:81:98:2E:DE:F6:8C:6E:…",1
"""EE:2E:A4:2A:68:FF:0D:62:2C:6A:…",1
"""01:89:B5:E5:C9:F1:7B:4C:5E:0D:…",1
"""60:24:CB:97:34:04:09:02:8B:75:…",1
…,…
"""E3:6C:DB:76:87:28:1D:D9:C0:1B:…",1
"""D9:FA:4E:C3:0A:97:E1:9D:FA:DF:…",1
"""A6:AC:12:41:A4:46:A0:D2:15:A6:…",1


A count of the different issuers, and their population percentage

In [21]:
total_counts = (
    df.group_by("issuer")
    .agg(pl.len().alias("total"))
    .with_columns((pl.col("total") / len(df) * 100).alias("percent"))
)
total_counts.sort("total", descending=True).head(10)

issuer,total,percent
str,u32,f64
"""Let's Encrypt Authority X3""",2241499,76.538113
"""cPanel, Inc. Certification Aut…",223862,7.643981
"""CloudFlare Inc ECC CA-2""",151667,5.178814
"""Sectigo RSA Domain Validation …",77891,2.659662
"""Fake LE Intermediate X1""",36528,1.247283
"""COMODO ECC Domain Validation S…",31234,1.066515
"""Go Daddy Secure Certificate Au…",24786,0.846342
"""CloudFlare Inc RSA CA-1""",23639,0.807176
"""GTS CA 1O1""",22482,0.767669


It'll be easier to work with the domains if we unnest them from their `all_domains` column

In [22]:
domains = df.explode(pl.col("all_domains")).rename({"all_domains": "full_domain"})

In [23]:
domains.sample(10)

fingerprint,full_domain,CN,issuer
str,str,str,str
"""44:01:79:CC:CF:33:8C:C0:F1:A8:…","""ods-observatoire.lab-123.com""","""ods-observatoire.org""","""cPanel, Inc. Certification Aut…"
"""B2:76:B5:0A:12:BB:E7:90:C3:D4:…","""www.acaiccristalina.com.br""","""acaiccristalina.com.br""","""Let's Encrypt Authority X3"""
"""1A:D7:93:B6:5F:1B:02:09:75:DA:…","""www.kidsyogaparty.com""","""kidsyogaparty.com""","""Let's Encrypt Authority X3"""
"""BA:FA:7A:DD:3F:73:6C:44:F7:55:…","""webmail.ktc613.com""","""ktc613.com""","""Let's Encrypt Authority X3"""
"""C5:8D:73:28:9A:15:E1:C4:DE:A8:…","""*.alannaysq.tk""","""sni162793.cloudflaressl.com""","""COMODO ECC Domain Validation S…"
"""E1:75:70:8C:70:35:13:2D:0F:F5:…","""www.havilahotelraftevold.no""","""havilahotelraftevold.no""","""Let's Encrypt Authority X3"""
"""B5:25:E1:A9:DD:4C:DC:5A:30:D1:…","""grafana.23wwc.online""","""grafana.23wwc.online""","""Let's Encrypt Authority X3"""
"""9A:BB:3C:4F:E3:19:07:2D:3D:3E:…","""webdisk.pokojispokoj.com.pl""","""pokojispokoj.com.pl""","""cPanel, Inc. Certification Aut…"
"""19:67:4A:CE:5B:64:20:E3:89:D9:…","""autodiscover.barbearia1950.com…","""barbearia1950.com.br""","""Let's Encrypt Authority X3"""


We'll split the `full_domain` up into multiple columns: the TLD, domain, and the subdomain

Instead of writing our own regex, we can leverage [tldextract](https://github.com/john-kurkowski/tldextract) to do the heavy lifting for us.

Note from the README:

> Beware when first calling tldextract, it updates its TLD list with a live HTTP request. This updated TLD set is usually cached indefinitely in `$HOME/.cache/python-tldextract`

I'll make an initial call to `tldextract` so that the HTTP request isn't called when we're using it with our dataframe.

Also leaving this here as a reminder to myself to remove that cache later

In [24]:
extract("http://forums.bbc-login.co.uk/")

ExtractResult(subdomain='forums', domain='bbc-login', suffix='co.uk', is_private=False)

In [25]:
domains = domains.with_columns(
    pl.col("full_domain")
    .map_elements(lambda x: extract(x).suffix, return_dtype=pl.String)
    .alias("TLD"),
    pl.col("full_domain")
    .map_elements(lambda x: extract(x).domain, return_dtype=pl.String)
    .alias("domain"),
    pl.col("full_domain")
    .map_elements(lambda x: extract(x).subdomain, return_dtype=pl.String)
    .alias("subdomain"),
)

In [26]:
domains.sample(10)

fingerprint,full_domain,CN,issuer,TLD,domain,subdomain
str,str,str,str,str,str,str
"""05:F4:5B:F8:0C:2A:82:9C:C1:4A:…","""webmail.highr.ca""","""highr.ca""","""Let's Encrypt Authority X3""","""ca""","""highr""","""webmail"""
"""8E:3E:C9:BB:C7:2F:FF:15:BE:1B:…","""webmail.americanskullery.com""","""iloveparakeets.com""","""Let's Encrypt Authority X3""","""com""","""americanskullery""","""webmail"""
"""3B:10:DE:18:B8:5A:24:8A:26:E0:…","""kebabelcarmenjaen.com""","""kebabelcarmenjaen.com""","""Let's Encrypt Authority X3""","""com""","""kebabelcarmenjaen""",""""""
"""D7:19:13:DE:43:B0:01:38:0B:15:…","""www.ubben-busvermietung.de""","""www.ubben-busvermietung.de""","""Let's Encrypt Authority X3""","""de""","""ubben-busvermietung""","""www"""
"""D9:79:A0:A3:E0:E3:4C:C4:0C:57:…","""webserver.naskot.fr""","""webserver.naskot.fr""","""Let's Encrypt Authority X3""","""fr""","""naskot""","""webserver"""
"""08:09:E6:27:2C:9C:69:33:F8:FE:…","""www.clinicadramigo.com.br""","""clinicadramigo.com.br""","""Let's Encrypt Authority X3""","""com.br""","""clinicadramigo""","""www"""
"""CE:E2:30:09:7F:71:14:60:6E:8B:…","""tachi-intl.com""","""tachi-intl.com""","""Let's Encrypt Authority X3""","""com""","""tachi-intl""",""""""
"""72:E0:E0:5C:C9:2C:46:A3:21:40:…","""cpanel.767sf.mx""","""767sf.mx""","""Let's Encrypt Authority X3""","""mx""","""767sf""","""cpanel"""
"""03:EA:B6:6B:F7:54:90:E3:13:29:…","""*.uptownbuymybag.com""","""sni194950.cloudflaressl.com""","""COMODO ECC Domain Validation S…","""com""","""uptownbuymybag""","""*"""


As you mentioned, one way we can try and find phishing domains is by taking a frequently exploited service or keyword and search for ones with a small edit-distance.

I'll use [another library](https://github.com/roy-ht/editdistance) for this; we could write our own edit-distance function, but this is written in C++ with Cython, so it's probably more performant when aggregrating over our dataframe

In [27]:
def check_edit_distance(x):
    keywords = ["apple", "paypal", "facebook", "twitter"]
    return any(editdistance.eval(x, keyword) < 2 for keyword in keywords)


domains.filter(
    pl.col("domain").map_elements(check_edit_distance, return_dtype=pl.Boolean)
).select("full_domain").sample(20).to_series().to_list()

['facebok.space',
 '*.twittev.info',
 'facabook.men',
 '*.apply.agency',
 'blog.facebook.web.id',
 'facebook.web.id',
 'autodiscover.paynal.com',
 'webdisk.facebook.web.id',
 'pacebook.co',
 'cpanel.pple.io',
 '*.paypl.xyz',
 'paipal.com.br',
 'www.facebolk.nl',
 'volumepurchaseprogram-uat-b.apple.com',
 'pacebook.co',
 '*.apply.agency',
 'cpanel.appe.org.br',
 'ample.in',
 'apply.agency',
 'myaccess-mdn.apple.com']

We can also search for highly nested keywords (in the subdomain)

In [28]:
domains.filter(
    (pl.col("subdomain").str.count_matches(r"\.") > 2)
    & (pl.col("subdomain").str.contains_any(["apple", "paypal", "facebook", "twitter"]))
).select("full_domain").sample(20).to_series().to_list()

['appleid.apple.com-securee-webvalid.com.ardianosta.com',
 'www.myaccountpaypal.summary.verificationclock.startxpo.com',
 'admin.www.facebook33.com.serveo.net',
 'mail.support.appleid.apple.authenticationkeyidmsa.com',
 'www.service.paypal.com.checkout30.com',
 '*.stats.paypal.com.admin-us.cas.ms',
 'www.facebook.23.com.serveo.net',
 'mail.peterbanksphotoworks.co.uk.pineapple.temporarywebsiteaddress.com',
 '*.work-16314996.facebook.com.rs2.cas.ms',
 'paypal.com.activation.account.verification.lcsimportexport.com.br',
 'manage.apple.com-us.secureaccesslock-pageunlock343.cetrukl.com',
 'www.iforgot.apple.com.locations-log.in',
 '*.mcastest9.facebook.com.us.cas.ms',
 'www.secure.paypal.com.aussierulesrebirth.com',
 'www.appleid.apple.ca.ios-icloud.net',
 'mail.wvw.support.apple.eksentrisitasreply.net',
 '*.ak.facebook.com.us2.cas.ms',
 'static.face.m.facebook.com.facebookcom.co.uk',
 'al.www.facebook30.com.serveo.net',
 'www.apple.com-payment.cancelnow.cancelorderapple.com']

We can put both of these together to come up with a small sample of potential phishing domains:

In [29]:
sample = domains.filter(
    (pl.col("domain").map_elements(check_edit_distance, return_dtype=pl.Boolean))
    | (
        (pl.col("subdomain").str.count_matches(r"\.") > 2)
        & (
            pl.col("subdomain").str.contains_any(
                ["apple", "paypal", "facebook", "twitter"]
            )
        )
    )
)

In [30]:
sample.shape

(539, 7)

In [31]:
sample.select("fingerprint").n_unique()

270

Certainly some suspicious domains:

In [32]:
sample.select("full_domain").sample(20).to_series().to_list()

['www.3apple.ru',
 'www.appleid.apple.com-onlyaccount.bppqlah.org',
 'www.appleid.apple.ca.ios-icloud.net',
 'cpanel.support.appleid.apple.authenticationkeyidmsa.com',
 '*.www.vupload-edge.facebook.com.rs2.cas.ms',
 'alterwind.www.facebookjesus.com.serveo.net',
 '*.dapple.com.my',
 'webmail.sapple.co.in',
 'peterbanksphotoworks.co.uk.pineapple.temporarywebsiteaddress.com',
 'myaccess-as-a.corp.apple.com',
 'www.myaccountpaypal.summary.verificationclock.startxpo.com',
 'paypl.xyz',
 'cpanel.updatepaypalaccountlimited.paypall.cc',
 'www.infos.appleprotecte.fr-vweb.ni2915540-3.web20.nitrado.hosting',
 'adkit.www.facebook1.com.serveo.net',
 'mail.appe.org.br',
 '*.apply.in.th',
 'mail.wvw.support.apple.eksentrisitasreply.net',
 'syobunapp.appli.online',
 '*.workplace.facebook.com.us.cas.ms']

Counts of each TLD from our sample:

In [33]:
sample.group_by("TLD").agg(pl.len().alias("count")).sort(
    by="count", descending=True
).head(10)

TLD,count
str,u32
"""com""",138
"""ms""",86
"""net""",72
"""web.id""",20
"""co.uk""",18
"""io""",16
"""fr""",15
"""org.br""",14
"""srv.br""",14


Counts of each issuer to these suspicious domains.

When aggregating, we take the count of the unique fingerprints to count the number of certificates that were issued with suspicious domains (an authority could have issued a certificate with multiple suspicious domains; we're counting this as 1, rather than 1 for each domain)

In [34]:
issuers_suspicious_count = sample.group_by("issuer").agg(
    pl.col("fingerprint").n_unique().alias("suspicious_count")
)
issuers_suspicious_count.sort(by="suspicious_count", descending=True).head(10)

issuer,suspicious_count
str,u32
"""Let's Encrypt Authority X3""",135
"""cPanel, Inc. Certification Aut…",44
"""COMODO ECC Domain Validation S…",24
"""Microsoft IT TLS CA 2""",18
"""Microsoft IT TLS CA 1""",15
"""Microsoft IT TLS CA 5""",14
"""Microsoft IT TLS CA 4""",10
"""DigiCert SHA2 Extended Validat…",4
"""CloudFlare Inc ECC CA-2""",3


We see that `Let's Encrypt Authority X3` has a lot, but this could also be because they constituted ~76% of the original data.

For better context, it's worth taking these counts and seeing how they compare to their total.

Filter out those with a `suspicious_count` of 1 to block out noise



In [35]:
res = (
    total_counts.join(issuers_suspicious_count, on="issuer", how="left", coalesce=True)
    .fill_null(0)
    .with_columns((pl.col("suspicious_count") / pl.col("total") * 100).alias("sus_pct"))
)
res.filter(pl.col("suspicious_count") > 1).sort(by=pl.col("sus_pct"), descending=True)

issuer,total,percent,suspicious_count,sus_pct
str,i64,f64,i64,f64
"""DigiCert SHA2 Extended Validat…",127,0.004337,4,3.149606
"""Microsoft IT TLS CA 2""",3524,0.12033,18,0.510783
"""Microsoft IT TLS CA 1""",3554,0.121355,15,0.42206
"""Microsoft IT TLS CA 5""",3508,0.119784,14,0.399088
"""Microsoft IT TLS CA 4""",3430,0.117121,10,0.291545
"""COMODO ECC Domain Validation S…",31234,1.066515,24,0.076839
"""cPanel, Inc. Certification Aut…",223862,7.643981,44,0.019655
"""Let's Encrypt Authority X3""",2241499,76.538113,135,0.006023
"""CloudFlare Inc ECC CA-2""",151667,5.178814,3,0.001978


In [36]:
con.close()